In [3]:
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn import datasets
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
import numpy as np
import optuna
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

### XGBoost Optuna with feature selection of corrleation 0.1

In [23]:
fetal = pd.read_csv("fetal_health.csv")
X = fetal[corr[abs(corr['fetal_health']) > 0.10]['fetal_health'].index]
X = X.drop(['fetal_health'], axis=1).values
y = fetal['fetal_health'].values.ravel().astype(int) - 1
corr = fetal.corr()

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y)

X_train_sub, X_valid, y_train_sub, y_valid = \
    train_test_split(X_train, y_train, test_size=0.2, random_state=123, stratify=y_train)

In [25]:
print('Train/Valid/Test sizes:', y_train.shape[0], y_valid.shape[0], y_test.shape[0])

Train/Valid/Test sizes: 1488 298 638


### Optuna

In [26]:
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
import numpy as np
import optuna


def optimization_objective(trial, X_train, y_train, cv=5):

    
    params =  {
            "n_estimators": trial.suggest_categorical("n_estimators", [30, 50, 100, 300, 500, 1000]),
            "learning_rate": trial.suggest_categorical("learning_rate", [0.01, 0.03, 0.05, 0.1]),
            "lambda": trial.suggest_loguniform("lambda", 1e-8, 1.0),
            "alpha": trial.suggest_loguniform("alpha", 1e-8, 1.0),
    }
    

    cv_iterator = StratifiedKFold(n_splits=cv, shuffle=True, random_state=123)

    cv_scores = np.zeros(cv)
    for idx, (train_sub_idx, valid_idx) in enumerate(cv_iterator.split(X_train, y_train)):
        
        X_train_sub, X_valid = X_train[train_sub_idx], X_train[valid_idx]
        y_train_sub, y_valid = y_train[train_sub_idx], y_train[valid_idx]
        

        model = XGBClassifier(**params, random_state=123, use_label_encoder=False)
        
        model.fit(
            X_train_sub,
            y_train_sub,
            verbose=False,
            eval_set=[(X_valid, y_valid)],
            eval_metric="auc",
            early_stopping_rounds=100,
        )
        
        preds = model.score(X_valid, y_valid)
        
        cv_scores[idx] = preds

    return np.mean(cv_scores)

In [27]:
study = optuna.create_study(direction="maximize", study_name="XGBoost Classifier")

def func(trial):
    return optimization_objective(trial, X_train, y_train)

study.optimize(func, n_trials=50);





print(f"Best CV accuracy: {study.best_value:.5f}")
print("Best params:")
for key, value in study.best_params.items():
    print(f"\t{key}: {value}")

[I 2021-11-22 17:08:02,124] A new study created in memory with name: XGBoost Classifier
[I 2021-11-22 17:08:14,312] Trial 0 finished with value: 0.9448963912051159 and parameters: {'n_estimators': 500, 'learning_rate': 0.03, 'lambda': 0.021464989218407376, 'alpha': 0.020392179485930688}. Best is trial 0 with value: 0.9448963912051159.
[I 2021-11-22 17:08:15,985] Trial 1 finished with value: 0.9422095677129235 and parameters: {'n_estimators': 30, 'learning_rate': 0.1, 'lambda': 0.011349463756739153, 'alpha': 9.788671630686054e-08}. Best is trial 0 with value: 0.9448963912051159.
[I 2021-11-22 17:08:20,393] Trial 2 finished with value: 0.9422163469143335 and parameters: {'n_estimators': 100, 'learning_rate': 0.05, 'lambda': 2.2498336351372585e-08, 'alpha': 0.0046144170104013785}. Best is trial 0 with value: 0.9448963912051159.
[I 2021-11-22 17:08:29,516] Trial 3 finished with value: 0.946916593225318 and parameters: {'n_estimators': 500, 'learning_rate': 0.05, 'lambda': 0.048475533228118

Best CV accuracy: 0.94692
Best params:
	n_estimators: 500
	learning_rate: 0.05
	lambda: 0.048475533228118345
	alpha: 6.714055775278799e-08


In [28]:
model = XGBClassifier(**study.best_params, random_state=123, use_label_encoder=False)
model.fit(
    X_train,
    y_train,
    verbose=False,
    eval_set=[(X_test, y_test)],
    eval_metric="auc",
    early_stopping_rounds=100,
)



print(f"Training Accuracy: {model.score(X_train, y_train)*100:0.3f}%")
print(f"Test Accuracy: {model.score(X_test, y_test)*100:0.3f}%")

Training Accuracy: 99.328%
Test Accuracy: 93.887%
